In [1]:
import numpy as np
import pandas as pd
import mlflow

In [2]:
mlflow.set_tracking_uri('http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000')
mlflow.set_experiment('Exp 5 - ML Algos with HP Tuning')

<Experiment: artifact_location='s3://neeraj-first-bucket/870835495118995563', creation_time=1736945300249, experiment_id='870835495118995563', last_update_time=1736945300249, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import ADASYN
import optuna

c:\data\projects\comment_analysis\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2)

# vectorizer
ngram_range = (1,2)
max_features = 2000
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
X_train_trf = vectorizer.fit_transform(X_train)
X_test_trf = vectorizer.fit_transform(X_test)

# oversampling
adasyn = ADASYN(random_state=42)
X_train_trf, y_train = adasyn.fit_resample(X_train_trf, y_train)


In [6]:
def log_mlflow(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_Adasyn_Tfidf_Bigram")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algorithm", model_name)

        # model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # metrics
        accuracy = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        mlflow.log_metric("accoracy", accuracy)
        for label, metrics in class_report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [7]:
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)
    return accuracy_score(y_test, model.fit(X_train_trf, y_train).predict(X_test_trf))

In [8]:
def optuna_run_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_rf, n_trials=30)

    best_params = study.best_params
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], min_samples_leaf=best_params['min_samples_leaf'], min_samples_split=best_params['min_samples_split'])
    
    log_mlflow("rf", best_model, X_train_trf, y_train, X_test_trf, y_test)

In [9]:
optuna_run_experiment()

[I 2025-01-17 12:01:23,369] A new study created in memory with name: no-name-7da2b293-9bab-4f29-8248-ba117813387b
[I 2025-01-17 12:01:30,647] Trial 0 finished with value: 0.40720032728760397 and parameters: {'n_estimators': 187, 'max_depth': 13, 'min_samples_leaf': 16, 'min_samples_split': 19}. Best is trial 0 with value: 0.40720032728760397.
[I 2025-01-17 12:01:38,707] Trial 1 finished with value: 0.40733669712259646 and parameters: {'n_estimators': 174, 'max_depth': 12, 'min_samples_leaf': 6, 'min_samples_split': 9}. Best is trial 1 with value: 0.40733669712259646.
[I 2025-01-17 12:01:39,711] Trial 2 finished with value: 0.409109504977499 and parameters: {'n_estimators': 93, 'max_depth': 5, 'min_samples_leaf': 7, 'min_samples_split': 18}. Best is trial 2 with value: 0.409109504977499.
[I 2025-01-17 12:01:41,515] Trial 3 finished with value: 0.40856402563752897 and parameters: {'n_estimators': 85, 'max_depth': 12, 'min_samples_leaf': 20, 'min_samples_split': 15}. Best is trial 2 with 

🏃 View run rf_Adasyn_Tfidf_Bigram at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/870835495118995563/runs/8b5126d99ad148029bff281778a19aad
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/870835495118995563
